In [1]:
from pprint import pprint
from IPython.display import display, Markdown

from src.contract_utils import query_contract
from config import LCD_CLIENT

PASSPORT_CONTRACT_ADDRESS = 'bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h3tu3tsy4jk9xlsfzhxel'
TREASURY_CONTRACT_ADDRESS = 'bostrom182jzjwdyl5fw43yujnlljddgtrkr04dpd30ywp2yn724u7qhtaqstjzlcu'
GIFT_CONTRACT_ADDRESS = 'bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhvh7ucrfuqaev6xq7cpvek'

In [2]:
passport_amount = query_contract(query='{"last_portid": {}}', contract_address=PASSPORT_CONTRACT_ADDRESS)['data']['portid']
gift_config = query_contract(query='{"config": {}}', contract_address=GIFT_CONTRACT_ADDRESS)['data']
gift_state = query_contract(query='{"state": {}}', contract_address=GIFT_CONTRACT_ADDRESS)['data']
treasury_balance = int(LCD_CLIENT.bank.balance(address=TREASURY_CONTRACT_ADDRESS)[0].to_data()[0]['amount'])
allowance = query_contract(query='{"all_allowances": {}}', contract_address=TREASURY_CONTRACT_ADDRESS)['data']['allowances'][0]

pprint(gift_config)
pprint(gift_state)

{'allowed_native': 'boot',
 'coefficient_down': '7',
 'coefficient_up': '13',
 'initial_balance': '700000000000000',
 'owner': 'bostrom1xszmhkfjs3s00z2nvtn7evqxw3dtus6yr8e4pw',
 'passport': 'bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h3tu3tsy4jk9xlsfzhxel',
 'target_claim': '100000'}
{'claims': '29527',
 'coefficient': '11.967904916481177142',
 'current_balance': '579585591807772',
 'releases': '50983'}


In [3]:
initial_balance = int(gift_config['initial_balance'])
target_claim = int(gift_config['target_claim'])
current_balance = int(gift_state['current_balance'])
claims = int(gift_state['claims'])
allowance_balance = int(allowance['balance'][0]['amount'])
claimed_amount = initial_balance - current_balance
released_amount = int(1e14 - treasury_balance)
releasable_percentage = (claims * 100) // target_claim / 100
releasable_amount = int(claimed_amount * releasable_percentage) - released_amount

In [4]:
display(Markdown(f'''### Gift overview'''))
print(
    f'total amount:      {initial_balance/1e9:>7,.0f} GBOOT\n'
    f'claimed amount:    {claimed_amount/1e9:>7,.0f} GBOOT ({claimed_amount/initial_balance:.2%})\n'
    f'releasable amount: {releasable_amount/1e9:>7,.0f} GBOOT  ({releasable_amount/initial_balance:.2%})\n'
    f'released amount:   {released_amount/1e9:>7,.0f} GBOOT  ({released_amount/initial_balance:.2%})\n\n'
    f'number of passports:     {passport_amount:>9,}\n\n'
    f'number of target claims: {target_claim:>9,}\n'
    f'number of claims:        {claims:>9,} ({claims/target_claim:.2%})\n\n'
    f'treasury balance:  {treasury_balance/1e9:>7,.0f} GBOOT\n'
    f'allowance balance: {allowance_balance/1e9:>7,.0f} GBOOT'
    )

### Gift overview

total amount:      700,000 GBOOT
claimed amount:    120,414 GBOOT (17.20%)
releasable amount:  18,615 GBOOT  (2.66%)
released amount:    16,305 GBOOT  (2.33%)

number of passports:        38,440

number of target claims:   100,000
number of claims:           29,527 (29.53%)

treasury balance:   83,695 GBOOT
allowance balance:  17,695 GBOOT
